# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [39]:
# import numpy and pandas
import numpy as np
import pandas as pd
import scipy.stats as st

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents.

In [3]:
salaries = pd.read_csv('Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [4]:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [10]:
salaries.isnull()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,False,False,False,False,False,True,False,True
1,False,False,False,False,False,True,False,True
2,False,False,False,False,False,True,False,True
3,False,False,False,False,False,True,False,True
4,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...
33178,False,False,False,False,False,True,False,True
33179,False,False,False,False,False,True,False,True
33180,False,False,False,False,False,True,False,True
33181,False,False,False,False,False,True,False,True


Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [24]:
num_salary = salaries['Salary or Hourly'].value_counts()[0]
num_hourly = salaries['Salary or Hourly'].value_counts()[1]

print(f'the percentage of hourly employs is {num_hourly/(num_hourly + num_salary)}')
print(f'the percentage of salary employs is {num_salary/(num_hourly + num_salary)}')

the percentage of hourly employs is 0.24175029382515142
the percentage of salary employs is 0.7582497061748485


What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [26]:
salaries['Department'].value_counts()

Department
POLICE                   13414
FIRE                      4641
STREETS & SAN             2198
OEMC                      2102
WATER MGMNT               1879
AVIATION                  1629
TRANSPORTN                1140
PUBLIC LIBRARY            1015
GENERAL SERVICES           980
FAMILY & SUPPORT           615
FINANCE                    560
HEALTH                     488
CITY COUNCIL               411
LAW                        407
BUILDINGS                  269
COMMUNITY DEVELOPMENT      207
BUSINESS AFFAIRS           171
COPA                       116
BOARD OF ELECTION          107
DoIT                        99
PROCUREMENT                 92
INSPECTOR GEN               87
MAYOR'S OFFICE              85
CITY CLERK                  84
ANIMAL CONTRL               81
HUMAN RESOURCES             79
CULTURAL AFFAIRS            65
BUDGET & MGMT               46
ADMIN HEARNG                39
DISABILITIES                28
TREASURER                   22
HUMAN RELATIONS             

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [91]:
# filtering for hourly workers
hourly_workers = salaries[salaries['Salary or Hourly']=='Hourly']

# sampling for n = 50
n = 50
hourly_workers_sample = hourly_workers.sample(50)
hourly_workers_sample.head()

#setting the test
mean = hourly_workers_sample['Hourly Rate'].mean()
std = hourly_workers_sample['Hourly Rate'].std(ddof = 1)
alpha = 0.05
mu = 30
stat = (mean - mu) / (std/np.sqrt(n))

#get p-value
p_value = st.t.sf(abs(stat), n-1) * 2
if p_value > alpha:
    print(f'our hypothesis is correct. p-values is {p_value}')
else:
    print(f"our hypothesis wasn't correct. p-values is {p_value}")

34.527
our hypothesis wasn't correct. p-values is 0.01742522954491291


We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [58]:
# filtering for hourly workers
salary_workers = salaries[salaries['Salary or Hourly']=='Salary']

# sampling for n = 50
n = 50
salary_workers_sample = salary_workers.sample(n)
salary_workers_sample.head()

#setting the test
mean = salary_workers_sample['Annual Salary'].mean()
std = salary_workers_sample['Annual Salary'].std(ddof = 1)
alpha = 0.05
mu = 86000
stat = (mean - mu) / (std/np.sqrt(n))

#get p-value
p_value = st.t.sf(abs(stat), n-1) * 2
p_value

if p_value > alpha:
    print(f'our hypothesis is correct. p-values is {p_value}')
else:
    print(f"our hypothesis wasn't correct. p-values is {p_value}")

84330.36
our hypothesis wasn't correct. p-values is 0.5550077500925089


Using the `crosstab` function, find the department that has the most hourly workers. 

In [86]:
#filtering df foi hourly workers
hourly_workers = salaries[salaries['Salary or Hourly']=='Hourly']

#counting the number of hourly workers by dpt
count_hourly = hourly_workers.groupby(['Department']).agg({'Salary or Hourly':'count'})

#sorting by descending order and finding what's the first dpt
count_hourly_desc = count_hourly.sort_values(by = 'Salary or Hourly', ascending = False)

dpt = count_hourly_desc.reset_index()['Department'][0]

print(f'the dpt with more hourly workers is {dpt}')

the dpt with more hourly workers is STREETS & SAN


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [87]:
# filtering for hourly workers
streets_workers = salaries[salaries['Department']=='STREETS & SAN']

# sampling for n = 50
n = 50
streets_workers_sample = streets_workers.sample(n)
streets_workers_sample.head()

#setting the test
mean = streets_workers_sample['Hourly Rate'].mean()
std = streets_workers_sample['Hourly Rate'].std(ddof = 1)
alpha = 0.05
mu = 35
stat = (mean - mu) / (std/np.sqrt(n))

#get p-value
p_value = st.t.sf(abs(stat), n-1) * 2
p_value

if p_value > alpha:
    print(f'our hypothesis is correct. p-values is {p_value}')
else:
    print(f"our hypothesis wasn't correct. p-values is {p_value}")

our hypothesis is correct. p-values is 0.1405739836240176


# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [122]:
#gathering all hourly wages
hourly_rates = salaries[salaries['Salary or Hourly'] == 'Hourly']['Hourly Rate']
n = 100
hourly_rates_sample = hourly_rates.sample(n)

#gathering variables
hr_mean = hourly_rates_sample.mean()
std = hourly_rates.std(ddof=0)

# calculating conf interval
f'confidence interval for 95%: {st.norm.interval(0.95, loc=hr_mean, scale=std/np.sqrt(n))}'

'confidence interval for 95%: (29.709927351854127, 34.45767264814585)'

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [124]:
#gathering all hourly wages
annual_salary = salaries[salaries['Salary or Hourly'] == 'Salary']['Annual Salary']
n = 100
annual_salary_sample = annual_salary.sample(n)

#gathering variables
as_mean = annual_salary_sample.mean()
std = annual_salary_sample.std(ddof=0)

# calculating conf interval
f'confidence interval for 95%: {st.norm.interval(0.95, loc=as_mean, scale=std/np.sqrt(n))}'


'confidence interval for 95%: (82776.67139237521, 91138.05900762479)'

# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [ ]:
# Your code here:

